[Notebook of task](https://github.com/DistributedSystemsGroup/Algorithmic-Machine-Learning/blob/master/Challenges/Plankton/plankton_challenge.ipynb)


# Classification of Plankton based on features 


In [5]:
!pip3 install --user 'mlens'

    100% |################################| 235kB 1.2MB/s 
You are using pip version 18.0, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [8]:
%matplotlib inline
# For configuration and jupiter
import os
import sys
import re
import random
import matplotlib
import implicit
import warnings
# For data manipulation
import pandas as pd
import numpy as np
# For visualization
import matplotlib.pyplot as plt
import seaborn as sns
# For prediction
from tqdm import tqdm

basepath = "/mnt/datasets/plankton/flowcam/"

In [9]:
nativeDF = pd.read_csv(basepath + 'features_native.csv.gz')
labelsDF = pd.read_csv(basepath + 'meta.csv')

In [10]:
#print(nativeDF.isnull().sum().sort_values(ascending=False)[:10])
#print(skimagDF.isnull().sum().sort_values(ascending=False)[:6])

native_nan_cols = ['perimareaexc', 'feretareaexc', 'cdexc', 'skeleton_area',
                   'nb1_area', 'symetrieh_area', 'symetriev_area', 'convarea_area',
                  'nb2_area', 'nb3_area', ]


In [11]:
for col_name in native_nan_cols:
    nativeDF[col_name] = nativeDF[col_name].fillna(0)

labelsDF['objid'] = labelsDF['objid'].astype(np.int64, errors='ignore')
labelsDF['level1'] = labelsDF['level1'].fillna('No_level1_name')
labelsDF['level2'] = labelsDF['level2'].fillna('No_level2_name')

print(nativeDF.isnull().sum().any())
print(labelsDF.isnull().sum().any())

False
False


In [12]:
y = labelsDF['level2'] 
X = nativeDF

In [13]:
from sklearn.model_selection import train_test_split

# Divide train data into train and validation sets

seed = 42
test_size = 0.20

X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size = test_size, random_state = seed)

In [19]:
# PCA performs best with a normalized feature set 
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()  
scX_train = sc.fit_transform(X_train)  
scX_test = sc.transform(X_test)  

In [20]:
from sklearn.decomposition import PCA

# 65 attributes in NativeDF
# 30 elements: 99,0%, 10 elements 83,4%, 20 elements: 96,1%

pca = PCA(.99)  
pcaX_train = pca.fit_transform(scX_train)  
pcaX_test = pca.transform(scX_test)  

In [14]:
from sklearn.metrics import accuracy_score, log_loss, f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

classifiers = [
    KNeighborsClassifier(3),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    MLPClassifier(alpha=1)
    ]

# Logging for Visual Comparison
log_cols=["Classifier", "Accuracy", "Log Loss", "F1 Macro Score"]
log = pd.DataFrame(columns=log_cols)


for clf in classifiers:
    clf.fit(X_train, y_train)
    name = clf.__class__.__name__
    
    train_predictions = clf.predict(X_test)
    acc = accuracy_score(y_test, train_predictions)
    
    f1 = f1_score(y_test, train_predictions, average='macro', labels=np.unique(train_predictions))
    
    train_predictions = clf.predict_proba(X_test)
    ll = log_loss(y_test, train_predictions)
    
    log_entry = pd.DataFrame([[name, acc*100, ll, f1]], columns=log_cols)
    log = log.append(log_entry)
    
log

,Classifier,Accuracy,Log Loss,F1 Macro Score
0,KNeighborsClassifier,52.920652,10.166362,0.108554
0,DecisionTreeClassifier,74.951767,8.651353,0.355398
0,RandomForestClassifier,81.562744,1.827556,0.453720
0,AdaBoostClassifier,48.312877,3.286494,0.229631
0,MLPClassifier,36.055581,22.061722,0.219377


In [22]:
# without PCA, RandomForestClassifer is best and KNeighbours and MLP is awful. With PCA, MLP and KNeighbours perform similar to DecisionTree and RandomForest

model = RandomForestClassifier(n_estimators=200, n_jobs=-1, criterion='entropy')
model.fit(X_train, y_train)
model_predictions = model.predict(X_test)
model_f1 = f1_score(y_test, model_predictions, average='macro', labels=np.unique(model_predictions))
print("Final F1 Score: ", model_f1)

Final F1 Score:  0.5051115100965339


In [22]:
# stacking 
from mlens.ensemble import Subsemble
from sklearn.linear_model import LogisticRegression
from mlens.metrics import make_scorer

rf = RandomForestClassifier(n_estimators=200, n_jobs=-1, criterion='entropy')
kn = KNeighborsClassifier(3)
dt = DecisionTreeClassifier()
mlpc = MLPClassifier(alpha=1)

lr = LogisticRegression()

seed = 42


f1 = make_scorer(f1_score, greater_is_better=True, average='macro', labels=np.unique(y_test))

ensemble = Subsemble(scorer = f1, random_state=seed, folds=10, verbose = 2)

# Build the first layer
ensemble.add([rf, kn, dt, mlpc])
# Attach the final meta estimator
ensemble.add_meta(lr)
ensemble.fit(X_train, y_train)
e_preds = ensemble.predict(X_test)

e_f1 = f1_score(y_test, e_preds, average='macro', labels=np.unique(e_preds))
print("Stacked F1 Score: ", e_f1)


Fitting 2 layers
Processing layer-1             

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/usr/lib/python3.5/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    179         sys.exit(msg)
    180     main_globals = sys.modules["__main__"].__dict__
    181     if alter_argv:
    182         sys.argv[0] = mod_spec.origin
    183     return _run_code(code, main_globals, None,
--> 184                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.5/dist-packages/ipykernel_launcher.py')
    185 
    186 def run_module(mod_name, init_globals=None,
    187                run_name=None, alter_sys=False):
    188     """Execute a module's code without importing it

...........................................................................
/usr/lib/python3.5/runpy.py in _run_code(code=<